In [1]:
import algorithm as al
import numpy as np
import pandas as pd

In [2]:
# train = pd.read_excel('../archive/Raisin_Dataset.xlsx')
train = pd.read_csv('../data/zoo/zoo.csv', header=None)
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [3]:
train = train.to_numpy()
R_train = train[:, 0:-1]
R_train
# R_train = np.random.rand(100, 10)

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 0, 0],
       ...,
       [1, 0, 0, ..., 1, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 0, 0]], dtype=int64)

In [4]:
M, N, H = al.method1(R_train, 0.45)

M = np.around(M, 3)
N = np.around(N, 3)
H = np.around(H, 3)

print ("M = ",M)
print ("\nN = ",N)
print ("\nH = ",H)

M =  [[0.016 0.017 0.006 ... 0.006 0.035 0.014]
 [0.019 0.018 0.006 ... 0.007 0.04  0.016]
 [0.008 0.02  0.008 ... 0.006 0.027 0.009]
 ...
 [0.02  0.018 0.006 ... 0.007 0.037 0.017]
 [0.009 0.022 0.008 ... 0.005 0.029 0.008]
 [0.009 0.048 0.009 ... 0.006 0.032 0.009]]

N =  [[0.69  0.678 0.79  ... 0.79  0.575 0.707]
 [0.662 0.669 0.789 ... 0.777 0.55  0.689]
 [0.761 0.66  0.762 ... 0.792 0.615 0.754]
 ...
 [0.657 0.67  0.788 ... 0.775 0.565 0.679]
 [0.756 0.642 0.763 ... 0.804 0.602 0.76 ]
 [0.758 0.518 0.757 ... 0.791 0.588 0.758]]

H =  [[0.295 0.305 0.204 ... 0.204 0.39  0.28 ]
 [0.319 0.313 0.205 ... 0.216 0.41  0.295]
 [0.231 0.32  0.229 ... 0.202 0.358 0.237]
 ...
 [0.323 0.312 0.206 ... 0.218 0.398 0.304]
 [0.235 0.335 0.229 ... 0.191 0.369 0.231]
 [0.234 0.433 0.235 ... 0.203 0.38  0.234]]


In [5]:
# Chuẩn bị dữ liệu cho phân cụm
X = np.dstack((M, N, H))
P = X.shape[0]
true_labels = train[:, -1]

# Danh sách các hàm độ đo khoảng cách
distance_measures = {
    'distance_function': al.distance_function,
    'hamming_distance': al.hamming_distance,
    'euclidean_distance': al.euclidean_distance,
    'normalized_euclidean_distance': al.normalized_euclidean_distance,
    'hausdorff_distance': al.hausdorff_distance,
    'yang_chiclana_distance': al.yang_chiclana_distance,
    'wang_xin_distance': al.wang_xin_distance,
    'liu_jiang_distance': al.liu_jiang_distance,
    'he_distance': al.he_distance,
    'thao_distance': al.thao_distance,
    'mahanta_panda_distance': al.mahanta_panda_distance,
}

# Vòng lặp qua các độ đo
for measure_name, measure_func in distance_measures.items():
    # Phân cụm sử dụng độ đo hiện tại
    U, S = al.method3(R_train, len(np.unique(train[:, -1])), 1.7, 0.45, measure_name)

    # Gán nhãn dựa trên khoảng cách tới các tâm cụm
    y = np.array([np.argmin([measure_func(X[i], S[j]) for j in range(S.shape[0])]) for i in range(P)])

    # Tính toán các chỉ số đánh giá
    CA = round(al.calculate_CA(true_labels, y), 2)
    PC = round(al.calculate_PC(U), 2)
    SC = round(al.calculate_SC(X, U, S, 1.7), 2)
    XB = round(al.calculate_XB(X, U, S), 2)
    DI = round(al.calculate_DI(X, U), 2)

    # In ra các chỉ số đánh giá cho mỗi độ đo
    print(f'Measure: {measure_name}')
    print(f'  Clustering Accuracy (CA): {CA}')
    print(f'  Partition Coefficient (PC): {PC}')
    print(f'  Partition Index (SC): {SC}')
    print(f'  Xie-Beni Index (XB): {XB}')
    print(f'  Dunn Index (DI): {DI}')
    print('------------------------------------------------')

Measure: distance_function
  Clustering Accuracy (CA): 0.04
  Partition Coefficient (PC): 0.76
  Partition Index (SC): 0.01
  Xie-Beni Index (XB): 4.83
  Dunn Index (DI): 0.22
------------------------------------------------
Error calculating DI: zero-size array to reduction operation maximum which has no identity
Measure: hamming_distance
  Clustering Accuracy (CA): 0.01
  Partition Coefficient (PC): 0.29
  Partition Index (SC): 0.02
  Xie-Beni Index (XB): 5.94181525215958e+17
  Dunn Index (DI): 0
------------------------------------------------
Measure: euclidean_distance
  Clustering Accuracy (CA): 0.17
  Partition Coefficient (PC): 0.36
  Partition Index (SC): 0.01
  Xie-Beni Index (XB): 1.2636149631209535e+23
  Dunn Index (DI): 0.25
------------------------------------------------
Error calculating DI: zero-size array to reduction operation maximum which has no identity
Measure: normalized_euclidean_distance
  Clustering Accuracy (CA): 0.37
  Partition Coefficient (PC): 0.23
  Par

In [6]:
M, N, H = al.method2_1(R_train, 0, 1, 0.45, 1)

M = np.around(M, 3)
N = np.around(N, 3)
H = np.around(H, 3)

# Chuẩn bị dữ liệu cho phân cụm
X = np.dstack((M, N, H))
P = X.shape[0]
true_labels = train[:, -1]

# Danh sách các hàm độ đo khoảng cách
distance_measures = {
    'distance_function': al.distance_function,
    'hamming_distance': al.hamming_distance,
    'euclidean_distance': al.euclidean_distance,
    'normalized_euclidean_distance': al.normalized_euclidean_distance,
    'hausdorff_distance': al.hausdorff_distance,
    'yang_chiclana_distance': al.yang_chiclana_distance,
    'wang_xin_distance': al.wang_xin_distance,
    'liu_jiang_distance': al.liu_jiang_distance,
    'he_distance': al.he_distance,
    'thao_distance': al.thao_distance,
    'mahanta_panda_distance': al.mahanta_panda_distance,
}

# Vòng lặp qua các độ đo
for measure_name, measure_func in distance_measures.items():
    # Phân cụm sử dụng độ đo hiện tại
    U, S = al.method3_1(R_train, len(np.unique(train[:, -1])), 1.7, 0.45, measure_name)

    # Gán nhãn dựa trên khoảng cách tới các tâm cụm
    y = np.array([np.argmin([measure_func(X[i], S[j]) for j in range(S.shape[0])]) for i in range(P)])

    # Tính toán các chỉ số đánh giá
    CA = round(al.calculate_CA(true_labels, y), 2)
    PC = round(al.calculate_PC(U), 2)
    SC = round(al.calculate_SC(X, U, S, 1.7), 2)
    XB = round(al.calculate_XB(X, U, S), 2)
    DI = round(al.calculate_DI(X, U), 2)

    # In ra các chỉ số đánh giá cho mỗi độ đo
    print(f'Measure: {measure_name}')
    print(f'  Clustering Accuracy (CA): {CA}')
    print(f'  Partition Coefficient (PC): {PC}')
    print(f'  Partition Index (SC): {SC}')
    print(f'  Xie-Beni Index (XB): {XB}')
    print(f'  Dunn Index (DI): {DI}')
    print('------------------------------------------------')

Measure: distance_function
  Clustering Accuracy (CA): 0.0
  Partition Coefficient (PC): 0.56
  Partition Index (SC): 0.02
  Xie-Beni Index (XB): 0.35
  Dunn Index (DI): 0.36
------------------------------------------------
Error calculating DI: zero-size array to reduction operation maximum which has no identity
Measure: hamming_distance
  Clustering Accuracy (CA): 0.08
  Partition Coefficient (PC): 0.14
  Partition Index (SC): 1.2800417294825458e+17
  Xie-Beni Index (XB): 2.2889725057139634e+21
  Dunn Index (DI): 0
------------------------------------------------
Error calculating DI: zero-size array to reduction operation maximum which has no identity
Measure: euclidean_distance
  Clustering Accuracy (CA): 0.0
  Partition Coefficient (PC): 0.16
  Partition Index (SC): 0.34
  Xie-Beni Index (XB): 2.7956277781742173e+26
  Dunn Index (DI): 0
------------------------------------------------
Error calculating DI: zero-size array to reduction operation maximum which has no identity
Measur